In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackathon-data/data_not_clean.pkl
/kaggle/input/hackathon-data-clear/data.pkl


In [2]:
! pip install transformers sentencepiece

In [3]:
import random
import numpy as np
import torch
from torch import nn
from collections import OrderedDict
from transformers import AutoTokenizer, AutoModel

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed = 42
set_seed(seed)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# Hyperparameters
num_epochs = 5
batch_size = 64
learning_rate = 0.001

In [7]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
        self.base = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
        n_dim = 312
        self.head = nn.Sequential(OrderedDict( [('fc_1' , nn.Linear(n_dim, n_dim*2)),
                                                ('relu_1' , nn.GELU()),
                                                ('layernorm_1' , nn.LayerNorm(n_dim*2, eps=1e-12)),
                                                ('fc_2' , nn.Linear(n_dim*2, n_dim)),
                                                ('relu_2' , nn.GELU()),
                                                ('dropout', torch.nn.Dropout(.0)),
                                                ('layernorm_2' , nn.LayerNorm(n_dim, eps=1e-12)),
                                                ('fc_3' , nn.Linear(n_dim, 2, bias=False))
                    ]))

    def forward(self, text):
        token = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
        model_output = self.base(**token)
        result = self.head(model_output.pooler_output)
        return result

In [8]:
# Unpickle dataset
df = pd.read_pickle('/kaggle/input/hackathon-data/data_not_clean.pkl')
df.head()

,query,text,label,clean_text,clean_query,embedding_text,embedding_query
0,Когда был спущен на воду первый миноносец «Спо...,Зачислен в списки ВМФ СССР 19 августа 1952 год...,1,Зачислен в списки ВМФ СССР 19 августа 1952 год...,Когда был спущен на воду первый миноносец «Спо...,"[-0.0030975677, -0.018142669, -0.0058722952, 0...","[0.00064380514, 0.0074218363, -0.03353223, -0...."
1,Как долго существовало британское телевизионно...,"Хрустальный лабиринт (""The Crystal Maze"") — бр...",1,"Хрустальный лабиринт (""The Crystal Maze"") — бр...",Как долго существовало британское телевизионно...,"[0.0026477594, -0.026646728, 0.0009579654, -0....","[-0.029795218, -0.01173853, -0.00032150946, -0..."
2,Когда родилась Князева Марина Леонидовна?,Князева Марина Леонидовна (род. 7 мая 1952 г.)...,1,Князева Марина Леонидовна (род. 7 мая 1952 г.)...,Когда родилась Князева Марина Леонидовна?,"[-0.036747612, -0.012604811, -0.0109199695, -0...","[-0.036575466, -0.010551005, -0.04117768, -0.0..."
3,Кто был главным художником мира Зен?,"В книге ""Half-Life 2: Raising the Bar"" художни...",1,"В книге ""Half-Life 2: Raising the Bar"" художни...",Кто был главным художником мира Зен?,"[-0.02514373, -0.023727695, -0.04738828, 0.011...","[-0.022960061, -0.013048667, -0.018877652, -0...."
4,Как звали предполагаемого убийцу Джона Кеннеди?,В 1966 году окружной прокурор Нового Орлеана Д...,1,В 1966 году окружной прокурор Нового Орлеана Д...,Как звали предполагаемого убийцу Джона Кеннеди?,"[0.0074619167, -0.024880972, -0.026498705, 0.0...","[-0.0124044325, -0.0020067864, -0.030558525, 0..."


In [9]:
# Train-test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split((df['clean_query']+' [SEP] '+df['clean_text']).to_numpy(), df['label'].to_numpy(), test_size=0.35, random_state=seed)

In [10]:
from torch.utils.data import Dataset

class PandasDataset(Dataset):
    def __init__(self, df):
        self.dataframe = df.reset_index()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        return list(self.dataframe.iloc[index])[1:]

In [11]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(PandasDataset(pd.DataFrame([X_train, y_train]).T),
                              batch_size=batch_size)
test_dataloader = DataLoader(PandasDataset(pd.DataFrame([X_test, y_test]).T),
                              batch_size=batch_size)
train_dataloader.dataset.__getitem__(0)

['Были ли женщины руководителями Триад Гонконга? [SEP] В 90-х годах «14К» считалась крупнейшей триадой в мире. В этот период под предводительством босса Ван Куок-коя по прозвищу «Сломанный зуб Кой», который руководил ответвлением «14К» в Макао, триада вела ожесточенную войну с конкурентами из «Шуйфонг». В 1999 году суд Макао приговорил Ван Куок-коя к 15-и годам тюремного заключения, а семеро его сообщников получили меньшие сроки . Спасаясь от полицейского прессинга, «14К» вышла за пределы Гонконга и обзавелась крепкими позициями в юго-восточном Китае, Америке и Европе, одновременно ещё больше уйдя в тень. В 2008 году члены «14К» оказались причастны к похищению китайской семьи с целью получения выкупа в Новой Зеландии (семью разыскали до того, как деньги были выплачены).',
 0]

In [12]:
model = Model().to(device)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# Uncomment to train model
# total_step = len(train_dataloader)
# for epoch in range(num_epochs):
#     for i, (texts, labels) in enumerate(train_dataloader):
#         # Forward pass
#         outputs = model(texts)
#         loss = criterion(outputs, labels)
        
#         # Backprpagation and optimizer step
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         if (i+1) % 100 == 0:
#             print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
#                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [15]:
# Uncomment to test the model
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for texts, labels in test_dataloader:
#         labels = labels.to(device)
#         outputs = model(texts)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f'Accuracy: {100 * correct / total} %')

# # Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')